# db.ipynb library module

### Provide sqlite database functionality for a front-end application.

**Code repository: [https://github.com/zero2cx/db-app-notebook](https://github.com/zero2cx/db-app-notebook)**

**Developer: [David Schenck](https://github.com/zero2cx/)**

**Part of [Building a Desktop Database Application](./Building a Desktop Database Application.ipynb)**

**App 5 - [The Python Mega Course](https://www.udemy.com/the-python-mega-course/) (Course Creator & Facilitator: [Ardit Sulce](http://pythonhow.com/author))**

Note: Python 3.6 or higher is required to execute this notebook application.

In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import sqlite3

### Class definiton: Database class

Open a connection to an existing database file, or instead create an empty database and connect to that.

Upon instantiation, keyword arguments are interpreted as the column names and data types for an Sqlite database table. If the table does not exist, then a new table is created. The database table's name mirrors the database's name.

In [3]:
class Database():

    def __init__(self, path, name, **kwargs):
        self.file = f'{path}/{name}.db'
        self.table = name
        self.columns = 1
        self.conn = sqlite3.connect(database=self.file)
        self.curs = self.conn.cursor()
        if kwargs:
            sql = f'CREATE TABLE IF NOT EXISTS {self.table} (id INTEGER PRIMARY KEY'
            for key in kwargs:
                self.columns += 1
                sql += f', {key.lower()} {kwargs[key]}'
            sql += ')'
            print(sql)
            self.curs.execute(sql)
            self.conn.commit()

    def __del__(self):
        '''
        To be executed upon destruction of the database instance.
        '''
        self.close()
        
    def to_html(self, head=False):
        '''
        Return a representation of the database as html-table elements.
        '''
        html = '<table>'
        if head:
            html = f'{html}<tr>'
            headers = self.get_column_names(True)
            for i in range(len(headers)):
                html = f'{html}<th>{headers[i]}</th>'
            html = f'{html}</tr>'
        for row in self.get_all_records():
            html = f'{html}<tr>'
            for i in range(len(row)):
                html = f'{html}<td>{row[i]}</td>'
            html = f'{html}</tr>'
        html = f'{html}</table>'
        return html
            
    def get_column_names(self, pkey=False):
        '''
        Return a list of the table's column names.
        
        The table's primary key is not returned, by default. When parameter
        pkey is True, the primary key is returned as well.
        '''
        sql = f'PRAGMA table_info({self.table})'
        columns = self.curs.execute(sql).fetchall()
        names = []
        for i in range(len(columns)):
            if pkey == False and columns[i][0] == 0:
                continue
            names.append(columns[i][1])
        return names

    def get_column_types(self, pkey=False):
        '''
        Return a list of the table's column data-types.
        
        The table's primary key is not returned, by default. When parameter
        pkey is True, the primary key is returned as well.
        '''
        sql = f'PRAGMA table_info({self.table})'
        columns = self.curs.execute(sql).fetchall()
        types = []
        for i in range(len(columns)):
            if pkey == False and columns[i][0] == 0:
                continue
            types.append(columns[i][2])
        return types

    def get_all_records(self):
        '''
        Return a list of all table records.
        '''
        sql = f'SELECT * FROM {self.table}'
        print(sql)
        return self.curs.execute(sql).fetchall()

    def get_records(self, **kwargs):
        '''
        Return a list of the database records that match criteria.
        '''
        sql = f'SELECT * FROM {self.table} WHERE'
        for key in kwargs:
            if not kwargs[key]:
                continue
            sql += f' {key}="{kwargs[key]}" AND'
        sql = sql[:-4]
        print(sql)
        return self.curs.execute(sql).fetchall()

    def add_record(self, record):
        '''
        Add a new database record.
        '''
        sql = f'INSERT INTO {self.table} VALUES (NULL'
        for field in record:
            sql += f', "{field}"'
        sql += ')'
        print(sql)
        self.curs.execute(sql)
        self.conn.commit()

    def delete_record(self, id):
        '''
        Delete the database record that corresponds with the passed 'id'.
        '''
        sql = f'DELETE FROM {self.table} WHERE id={id}'
        print(sql)
        self.curs.execute(sql)
        self.conn.commit()

    def update_record(self, id, **kwargs):
        '''
        Update the database record that corresponds with the passed 'id'.
        '''
        sql = f'UPDATE {self.table} SET ('
        for key in kwargs:
            sql += f'{key}, '
        sql = sql[:-2]
        sql += ') = ('
        for key in kwargs:
            sql += f'"{kwargs[key]}", '
        sql = sql[:-2]
        sql += f') WHERE id={id}'
        print(sql)
        self.curs.execute(sql)
        self.conn.commit()

    def close(self):
        '''
        Close the connection to the database.
        '''
        self.conn.close()

### TODO: add the capacity to create and administer multiple tables.

### Example usage: Database and table creation.

Sqlite queries are automatically printed to the console.

The dictionary 'inventory' contains details of column-names and -types.


In [4]:
from IPython.display import HTML

!rm -f ./data/inventory.db

inventory = {'shelf_item': 'TEXT', 'unit_price': 'REAL', 'count': 'INTEGER', 'restock': 'INTEGER'}

db = Database(path='./data', name='inventory', **inventory)

# For display purposes here, the resulting table's column-names and -types are represented as a list of 2-tuple pairs.
display(HTML('<strong>The result:</strong>'))
print(f'table columns: {list(zip(db.get_column_names(True), db.get_column_types(True)))}')

CREATE TABLE IF NOT EXISTS inventory (id INTEGER PRIMARY KEY, shelf_item TEXT, unit_price REAL, count INTEGER, restock INTEGER)


table columns: [('id', 'INTEGER'), ('shelf_item', 'TEXT'), ('unit_price', 'REAL'), ('count', 'INTEGER'), ('restock', 'INTEGER')]


### Example usage: Add some records to the database table.

Sqlite queries are automatically printed to the console.

The example data represents four grocery-store shelf items in a list of lists.

In [5]:
from IPython.display import HTML

items = [
    ['Canned Pears', 1.39, 4, 2],
    ['Lollipop', 0.45, 15, 6],
    ['Toothbrush', 2.89, 6, 8],
    ['ShamWow!', 12.99, 3, 3]
]

for item in items:
    db.add_record(record=item)

display(HTML('<strong>The result:</strong>'))
display(HTML(db.to_html(True)))

INSERT INTO inventory VALUES (NULL, "Canned Pears", "1.39", "4", "2")
INSERT INTO inventory VALUES (NULL, "Lollipop", "0.45", "15", "6")
INSERT INTO inventory VALUES (NULL, "Toothbrush", "2.89", "6", "8")
INSERT INTO inventory VALUES (NULL, "ShamWow!", "12.99", "3", "3")


SELECT * FROM inventory


id,shelf_item,unit_price,count,restock
1,Canned Pears,1.39,4,2
2,Lollipop,0.45,15,6
3,Toothbrush,2.89,6,8
4,ShamWow!,12.99,3,3


### Example usage: Delete a record from the database table.

Sqlite queries are automatically printed to the console.

In [6]:
from IPython.display import HTML

# Toothbrushes are off the menu.
db.delete_record(id=3)

display(HTML('<strong>The result:</strong>'))
display(HTML(db.to_html(True)))

DELETE FROM inventory WHERE id=3


SELECT * FROM inventory


id,shelf_item,unit_price,count,restock
1,Canned Pears,1.39,4,2
2,Lollipop,0.45,15,6
4,ShamWow!,12.99,3,3


### Example usage: Update a record in the database table.

Sqlite queries are automatically printed to the console.

In [7]:
from IPython.display import HTML

# Six new ShamWows! just arrived, so change the count.
db.update_record(id=4, **{'count': '9'})

display(HTML('<strong>The result:</strong>'))
display(HTML(db.to_html(True)))

UPDATE inventory SET (count) = ("9") WHERE id=4


SELECT * FROM inventory


id,shelf_item,unit_price,count,restock
1,Canned Pears,1.39,4,2
2,Lollipop,0.45,15,6
4,ShamWow!,12.99,9,3


### Example usage: Close the connection to the database.

In [8]:
# Do this.
db.close()

# Or this.
db = None

### Information about this notebook:

In [9]:
from IPython.display import FileLink

print('Additional notebooks in this application (click to open in a new tab):')
display(FileLink('Building a Desktop Database Application.ipynb'))
display(FileLink('seed.ipynb'))

license = '''
This software is licensed under the Gnu GPLv3
(c) 2017 David Schenck https://github.com/zero2cx/

This program is free software: you can redistribute it and/or modify
it under the terms of the GNU General Public License as published by
the Free Software Foundation, either version 3 of the License, or
(at your option) any later version.

This program is distributed in the hope that it will be useful,
but WITHOUT ANY WARRANTY; without even the implied warranty of
MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
GNU General Public License for more details.

You should have received a copy of the GNU General Public License
along with this program.  If not, see <https://www.gnu.org/licenses/>.
'''

Additional notebooks in this application (click to open in a new tab):


/home/zero2cx/projects/the-python-mega-course/App5/Building a Desktop Database Application.ipynb

/home/zero2cx/projects/the-python-mega-course/App5/seed.ipynb